In [24]:
from youtube_transcript_api import YouTubeTranscriptApi
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
import pickle
import random
from collections import defaultdict
from pymongo import MongoClient

# Scrape URLs

In [20]:
def get_video_links_ted(scroll = 5):
    
    chromedriver = '/Applications/chromedriver'
    os.environ['webdriver.chrome.driver'] = chromedriver
    
    driver = webdriver.Chrome(chromedriver)
    driver.get('https://www.youtube.com/user/TEDtalksDirector/videos')
    
    for i in range(scroll):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    contents_div = soup.find('div', id='contents')

    res = contents_div.find_all('a', 
                                attrs={'class': 'yt-simple-endpoint style-scope ytd-grid-video-renderer'})
    
    links = []
    
    for i in res:
        links.append(i.get('href').replace('/watch?v=', ''))
        
    return links
    

In [21]:
links_video_ted = get_video_links_ted(scroll = 100)

In [22]:
len(links_video_ted)

2730

In [23]:
with open('ted_video_links_2730.pkl', 'wb') as picklefile:
    pickle.dump(links_video_ted, picklefile)

# Load Transcripts to Mongo

In [29]:
client = MongoClient()
db = client['ted']
db.create_collection("ted_transcripts")
ted_transcripts = db.get_collection('ted_transcripts')

In [30]:
links_video_ted[1:5]

['bNmRr-BYnxA', 'FVUkKKc3Vvk', '8bj0GR34XWc', 'eaCrsBtiYA4']

In [31]:
for url in links_video_ted:
    try:
        response = YouTubeTranscriptApi.get_transcript(url)
        ted_transcripts.insert_one({'url': url, 'transcript': response})
    except:
        ted_transcripts.insert_one({'url': "--", 'transcript': "--"})

In [32]:
ted_transcripts.count()

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2730

In [33]:
ted_transcripts.find({'transcript': '--'}).count()

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


314

In [36]:
#list(ted_transcripts.find())